In [1]:
import pandas as pd
import re

In [2]:
import mysql.connector
db = mysql.connector.connect(user='root', password='password', database='folklore_db')
cur = db.cursor()

In [3]:
from numpy import nan
from collections import namedtuple
import time
from IPython.display import clear_output

from pymystem3 import Mystem
m = Mystem()

## Zlynkovsk2016-2018

### Собиратели

```
dkbrz@dkbrz:~/GitHub/folklore_corpus$ mdb-export -d "\t" /home/dkbrz/Downloads/Zlynkovsk2016-2018.mdb "tblCards" > ./csv/tblCards.csv
dkbrz@dkbrz:~/GitHub/folklore_corpus$ mdb-export -d "\t" /home/dkbrz/Downloads/Zlynkovsk2016-2018.mdb "tblSobirately" > ./csv/tblSobirately.csv
dkbrz@dkbrz:~/GitHub/folklore_corpus$ mdb-export -d "\t" /home/dkbrz/Downloads/Zlynkovsk2016-2018.mdb "tblKeywords" > ./csv/tblKeywords.csv
dkbrz@dkbrz:~/GitHub/folklore_corpus$ mdb-export -d "\t" /home/dkbrz/Downloads/Zlynkovsk2016-2018.mdb "tblInformators" > ./csv/tblInformators.csv
```

In [16]:
OneCollector = namedtuple('OneCollector', ['old_id', 'code', 'name'])

def parse_one_collector(row):
    old_id = int(row['Номер'])
    code = row['Код собирателя']
    name = row['ФИО собирателя']
    return OneCollector(old_id, code, name)

data = pd.read_csv('./csv/tblSobirately.csv', sep='\t')
parse_one_collector(data.iloc[0])

OneCollector(old_id=412, code='АБМ', name='Мороз Андрей Борисович')

In [17]:
collectors = [parse_one_collector(data.iloc[i]) for i in range(len(data))]

In [18]:
collectors[:2]

[OneCollector(old_id=412, code='АБМ', name='Мороз Андрей Борисович'),
 OneCollector(old_id=413, code='ВК', name='Комарова Вера Андреевна')]

In [19]:
for i in collectors[0]:
    print (i, type(i))

412 <class 'int'>
АБМ <class 'str'>
Мороз Андрей Борисович <class 'str'>


In [21]:
cur.executemany('INSERT INTO collectors (old_id, code, name) VALUES (%s, %s, %s)', collectors)
db.commit()

### Информанты

In [23]:
OnePerson = namedtuple('OnePerson', ['old_id', 'code', 'name', 'birth_year', 'gender', 
                                     'bio', 'current_village', 'birth_village'])

def g_name(name):
    global m
    res = m.analyze(name)
    fem, mal = 0, 0
    if len(res) == 6:
        res = res[2:]
    for i in res:
        if 'analysis' in i:
            try:
                if ',f,' in i['analysis'][0]['gr']:
                    fem += 1
                elif ',m,' in i['analysis'][0]['gr']:
                    mal += 1
            except:
                pass
    if fem > 0 and mal == 0:
        return 'f'
    elif fem == 0 and mal > 0:
        return 'm'
    else:
        return None

def parse_one_person(row):
    old_id = int(row['номер'])
    code = row['инициалы']
    name = row['ФИО']
    
    date = re.findall('([0-9]{4}) г..?р.', row['биография'])
    if len(date) > 0:
        birth_year = int(date[0])
    else: 
        birth_year = None
    
    gender = g_name(name)
    bio = row['биография']
    current_village = row['село'].strip()
    
    birthplace = re.findall('род.? в .{1,4}? (.*?)[.(,]', row['биография'])
    if len(birthplace) > 0:
        birth_village = birthplace[0].strip()
    else: 
        birth_village = None
    return OnePerson(old_id, code, name, birth_year, gender, bio, current_village, birth_village)

data = pd.read_csv('./csv/tblInformators.csv', sep='\t')
parse_one_person(data.iloc[0])

OnePerson(old_id=1, code='КГИ', name='Католикова Галина Ивановна', birth_year=1941, gender='f', bio='1941 г.р., род. в с. Рогов. В 1961 г. переехала в д. Кривуша (= Зеленая Роща). С 1971 г. живет в Малых Щербиничах. Работала в колхозе.', current_village='Малые Щербиничи', birth_village='Рогов')

In [24]:
informators = [parse_one_person(data.iloc[i]) for i in range(len(data))]

In [25]:
for i in informators[0]:
    print (i, type(i))

1 <class 'int'>
КГИ <class 'str'>
Католикова Галина Ивановна <class 'str'>
1941 <class 'int'>
f <class 'str'>
1941 г.р., род. в с. Рогов. В 1961 г. переехала в д. Кривуша (= Зеленая Роща). С 1971 г. живет в Малых Щербиничах. Работала в колхозе. <class 'str'>
Малые Щербиничи <class 'str'>
Рогов <class 'str'>


In [26]:
cur.executemany('INSERT INTO informators (old_id, code, name, birth_year, gender, bio, current_village, \
birth_village) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)', informators)
db.commit()

### Тексты

In [36]:
OneText = namedtuple('OneText', ['old_id', 'year', 'leader', 'region', 'district', 'village', 'raw_text', 
                                 'keywords', 'question_list','questions', 'informators', 'collectors'])

def restore_label(label):
    label = label.split(',')
    result = []
    for i in label:
        q = re.findall('^([0-9]*)[^0-9a-zа-я]*?([ a-zа-я\.]*?)$', i.strip())[0]
        result.append((q[0].strip(), q[1].strip()))
    for key, value in enumerate(result):
        if value[0] == '':
            result[key] = (result[key-1][0].strip(), value[1].strip())
    return [''.join(i) for i in result]

def text_prettify(text):
    text = text.replace('...','…')
    text = re.sub('\n*?(\[[^\[]*?:[^\[]*?])',r'\n\n\1', text)
    return text.strip()

def parse_one_row(row):
    leader = 'АБМ'
    old_id = str(row['номер'])
    year = int(row['год'])
    raw_text = text_prettify(row['текст'])
    region, district, village = tuple([i.strip() for i in row['село'].split(',')])
    try:
        keywords = [word.strip() for word in row['ключевые слова'].split(',')]
    except AttributeError:
        keywords = []
    question_list = row['программа'].strip()
    questions = restore_label(row['вопрос'])
    informators = []
    for i in row[['информант','информант2','информант3','информант4']].values:
        if i is not nan:
            for j in i.split(','):
                informators.append(j.strip())
    collectors = [i for i in row[['собиратель','собиратель2','собиратель3','собиратель4']].values if i is not nan]
    return OneText(old_id, year, leader, region, district, village, raw_text, keywords, question_list, questions, informators, collectors)

data = pd.read_csv('./csv/tblCards.csv', sep='\t')
parse_one_row(data.iloc[0])

OneText(old_id='1', year=2017, leader='АБМ', region='Брянская обл.', district='Злынковский р-н', village='Спиридонова Буда', raw_text='[Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\\ндыка».] [Ма\\ндыка?] \n\n[Г:] Да. [Что это значит?] \n\n[Г, смеясь:] Ну, манды\\ка — э\\то… \n\n[А одновременно:] Коро\\че её ёб… \n\n[Г:] Все. \n\n[А:] Всё село\\. \n\n[Г:] Да, коро\\че… \n\n[А:] И… и… и из на\\шего, и коро\\че там… \n\n[Г:] Все подря\\д. \n\n[А:] Да. \n\n[А и Г одновременно:] Кому\\ не лень. \n\n[А:] Буты\\лку на стол, и… [Она пьёт, да?] \n\n[Г:] Ну, она\\ по-чё\\рному пьёт.', keywords=['Водка', 'Плата', 'Питье', 'Девственность', 'Женщина', 'Половой акт', 'Зов', 'Прозвище', 'Имя', 'Дразнить', 'Пьянство', 'Черный цвет', 'Цвет'], question_list='XXIa', questions=['5б'], informators=['А', 'Г'], collectors=['МЭГ', 'НВП', 'СЕ', 'ФЕЕ'])

In [37]:
for i in range(len(data)):
    try:
        parse_one_row(data.iloc[i])
    except:
        print (data.iloc[i]['вопрос'])

In [38]:
parsed = [parse_one_row(data.iloc[i]) for i in range(len(data))]

In [39]:
parsed[0]

OneText(old_id='1', year=2017, leader='АБМ', region='Брянская обл.', district='Злынковский р-н', village='Спиридонова Буда', raw_text='[Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\\ндыка».] [Ма\\ндыка?] \n\n[Г:] Да. [Что это значит?] \n\n[Г, смеясь:] Ну, манды\\ка — э\\то… \n\n[А одновременно:] Коро\\че её ёб… \n\n[Г:] Все. \n\n[А:] Всё село\\. \n\n[Г:] Да, коро\\че… \n\n[А:] И… и… и из на\\шего, и коро\\че там… \n\n[Г:] Все подря\\д. \n\n[А:] Да. \n\n[А и Г одновременно:] Кому\\ не лень. \n\n[А:] Буты\\лку на стол, и… [Она пьёт, да?] \n\n[Г:] Ну, она\\ по-чё\\рному пьёт.', keywords=['Водка', 'Плата', 'Питье', 'Девственность', 'Женщина', 'Половой акт', 'Зов', 'Прозвище', 'Имя', 'Дразнить', 'Пьянство', 'Черный цвет', 'Цвет'], question_list='XXIa', questions=['5б'], informators=['А', 'Г'], collectors=['МЭГ', 'НВП', 'СЕ', 'ФЕЕ'])

In [40]:
texts = [i[:7] for i in parsed]
col = [i.collectors for i in parsed]
inf = [i.informators for i in parsed]
questions = [(i.question_list, i.questions,) for i in parsed]
keywords = [i.keywords for i in parsed]

In [41]:
texts[:2]

[('1',
  2017,
  'АБМ',
  'Брянская обл.',
  'Злынковский р-н',
  'Спиридонова Буда',
  '[Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\\ндыка».] [Ма\\ндыка?] \n\n[Г:] Да. [Что это значит?] \n\n[Г, смеясь:] Ну, манды\\ка — э\\то… \n\n[А одновременно:] Коро\\че её ёб… \n\n[Г:] Все. \n\n[А:] Всё село\\. \n\n[Г:] Да, коро\\че… \n\n[А:] И… и… и из на\\шего, и коро\\че там… \n\n[Г:] Все подря\\д. \n\n[А:] Да. \n\n[А и Г одновременно:] Кому\\ не лень. \n\n[А:] Буты\\лку на стол, и… [Она пьёт, да?] \n\n[Г:] Ну, она\\ по-чё\\рному пьёт.'),
 ('2',
  2017,
  'АБМ',
  'Брянская обл.',
  'Злынковский р-н',
  'Спиридонова Буда',
  '[Разговор о кусачей собаке:] \n\n[Г:] А раз её\\ э\\тот Жидо\\к погла\\дил, так ему\\ ру\\ку прокуси\\ла наскво\\зь. Всегда\\ к нему\\ так [?]. [Жидок?] Да, жидо\\к, у него\\ кли\\чка така\\я. [Кто это?] Рыба\\к наш. \n\n[А:] Да. Пе\\рвый на селе\\…. \n\n[Г:] Пе\\рвый на селе\\ рыба\\к. [Тот, что в фуражке?] \n\n[А и Г:] Нет. \n\n[Г:] Не… \n\n[А:] В фура\\

In [43]:
cur.executemany('INSERT INTO texts (old_id, year, leader, region, district, village, raw_text) \
VALUES (%s,%s,%s,%s,%s,%s,%s)', texts)
db.commit()

In [27]:
con.commit()

**Собиратели**

In [50]:
cur.execute('SELECT * FROM collectors')
d = {i[2]:i[0] for i in cur.fetchall()}

In [51]:
d

{'АБМ': 1,
 'ВК': 2,
 'ПН': 3,
 'НВП': 4,
 'СНА': 5,
 'ГМО': 6,
 'СЕ': 7,
 'МЭГ': 8,
 'САИ': 9,
 'ГАА': 10,
 'БТБ': 11,
 'ДГЮ': 12,
 'ЛМА': 13,
 'ВДК': 14,
 'ФДД': 15,
 'БОВ': 16,
 'ККК': 17,
 'ТМА': 18,
 'ЗАЛ': 19,
 'НАС': 20,
 'КАС': 21,
 'ОЕС': 22,
 'КУК': 23,
 'ИДД': 24,
 'ФЕЕ': 25,
 'КЕА': 26,
 'ЧМА': 27,
 'БСК': 28,
 'МВБ': 29}

In [55]:
for key, value in enumerate(parsed):
    try:
        a = [d[i] for i in value.collectors]
        for i in a:
            cur.execute('INSERT into t_c (id_text, id_collector) VALUES (%s, %s)', (key+1, i))
    except:
        pass

In [56]:
db.commit()

In [57]:
assert len(d) == 29

In [32]:
con.commit()

**Информанты**

In [58]:
cur.execute('SELECT * FROM informators')
seen = set()
d = {}
for i in cur.fetchall():
    if i[2] not in seen:
        d[i[2]] = i[0]
        seen.add(i[2])
    else:
        if i[2] in d:
            del d[i[2]]

In [59]:
len(d), len(seen)

(225, 237)

In [63]:
for key, value in enumerate(parsed):
    try:
        a = [d[i] for i in value.informators]
        for i in a:
            cur.execute('INSERT into t_i (id_text, id_informator) VALUES (%s, %s)', (key+1, i))
    except:
        pass

In [64]:
db.commit()

In [108]:
for i in d:
    cur.execute('UPDATE TextInformators SET informator_id = ? WHERE code = ?', (d[i], i,))

In [109]:
con.commit()

In [61]:
double = seen - (seen&set(d.keys()))

In [62]:
double

{'А', 'В', 'ВА', 'ДНВ', 'КАИ', 'КВА', 'КВИ', 'ЛП', 'МАН', 'С', 'ХВМ', 'ХТА'}

In [65]:
from collections import defaultdict

In [90]:
double_dict = defaultdict(dict)

In [91]:
cur.execute('SELECT id, code, current_village FROM informators ORDER BY id DESC')
data = cur.fetchall()

In [ ]:
data

In [92]:
for i in data:
    if i[1] in double:
        double_dict[i[1]][i[2]] = i[0]

In [93]:
double_dict = dict(double_dict)
double_dict

{'КВА': {'Рогов': 238, 'Спиридонова Буда': 131},
 'ЛП': {'Рогов': 225, 'Спиридонова Буда': 92},
 'А': {'Рогов': 220, 'Спиридонова Буда': 101},
 'В': {'Рогов': 206, 'Малые Щербиничи': 12},
 'С': {'Рогов': 189, 'Спиридонова Буда': 119},
 'ХВМ': {'Рогов': 187, 'Спиридонова Буда': 78},
 'ДНВ': {'Рогов': 181, 'Малые Щербиничи': 22},
 'МАН': {'Рогов': 178, 'Карпиловка': 114},
 'КВИ': {'Спиридонова Буда': 162, 'Малые Щербиничи': 10},
 'КАИ': {'Спиридонова Буда': 150, 'Кривуша': 59},
 'ВА': {'Спиридонова Буда': 139, 'Шурубовка': 54},
 'ХТА': {'Спиридонова Буда': 133, 'Шурубовка': 30}}

In [94]:
s = '(\''+'\', \''.join(double_dict.keys())+'\')'
s

"('КВА', 'ЛП', 'А', 'В', 'С', 'ХВМ', 'ДНВ', 'МАН', 'КВИ', 'КАИ', 'ВА', 'ХТА')"

In [103]:
for key, value in enumerate(parsed):
    try:
        for i in value.informators:
            if i in double_dict:
                if value.village in double_dict[i]:
                    cur.execute('INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)', 
                                (key+1, double_dict[i][value.village],))
                else:
                    print ('!!!')
    except:
        print ('!!!')

In [104]:
db.commit()

In [104]:
cur.executemany('UPDATE TextInformators SET informator_id = ? WHERE code = ? AND id_text = ?', text_data)

In [105]:
con.commit()

In [ ]:
for i in double_dict:
    for j in double_dict[i]:
        

In [8]:
cur.execute('SELECT DISTINCT word FROM keywords')
s = set()
for i in sorted(set([i[0] for i in cur.fetchall()])):
    s.add(i)

In [112]:
s = set()
for text in parsed:
    s.update(text.keywords)

In [106]:
words = pd.read_csv('./csv/tblKeywords.csv', header=0, sep='\t')

In [107]:
words.head(1)

,ключевые слова,код
0,Ад,3


In [121]:
for i in range(len(words)):
    cur.execute('INSERT INTO keywords (word, old_id) VALUES (%s, %s)', 
                (words.iloc[i].values[0], int(words.iloc[i].values[1]),))

In [122]:
db.commit()

In [113]:
s2 = set(words['ключевые слова'].values)

In [114]:
len(s - (s2&s))

354

In [118]:
df = pd.DataFrame({'bad':list(sorted(s - (s2&s)))})

In [117]:
print('\n'.join(sorted(s - (s2&s))))


Бегать
Безеза
Белый
Беременость
Беремнность
Берменность
Бессоница
Бить
Битье посуда
Благопжелания
Благословение
Благословление
Блеый цвет
Блин
Блохи
Болезь
Большой-маленкий
Брать-давать
Быличка
Варвара св.
Ввремя
Великий Пост
Верх-них
Весель
Веснушники
Ветка
Внешний-внутренний
Внуренний-внешний
Внутрений-внешний
Внутреннее-внешнее
Внутренни-внешний
Водко
Вокресенье (день недели)
Воскренье (день недели)
Воскресенье
Вреия
Всемирынй потоп
Втыкать
Выбрасивать
Выбрасывание
Выбросить
Говорит
Годы
Головая боль
Головной убор
Горорить
Грешный-праведник
Гулние
ДВеснадцать
Давать-брать
Давать-брать Три
Двать-брать
Двери
Демонологияя
Дерева
Деревно
Дерево Ветки
Дерецве обрядовое
Десткий фольклор
Детский дольклор
Детский фолькор
Детский фольлор
Дни нелели
Днмонология
Добро-Зло
Добрый-злой
Додь
Доп
Древо
Женский-Супругиской
Женщины
Животное Выгон скота
Животные
Жизнь-Смерть
Жизнь-смерть Память
Жизнь-сметь
Завязывние-развязывание
Загаворы
Заговор
Закрещевание
Закрещивать
Закрыть
Запрет
Звезда (рожд.

In [45]:
for i in range(len(words)):
    cur.execute('INSERT INTO Keywords (word, old_id) VALUES (?, ?)', (words.iloc[i].values[0], int(words.iloc[i].values[1]),))

In [46]:
con.commit()

# Вторая часть

In [128]:
informators = pd.read_csv('./csv/informators.tsv', sep='\t').fillna('')

In [129]:
informators.head()

,шифр информанта,ФИО полное,год рождения,место рождения,gender
0,АНВ-1958-НЛ-2018,Агеичева Нина Владиславовна,1958,г. Пестяки,f
1,АН-1940-Юж-2018,Альбина Николаевна,1940,,f
2,ДЛЮ-1958-МН-2018,Дикина Людмила Юрьевна,1958,,f
3,ЖЕА-1932-НЛ-2018,Жихарев Евгений Александрович,1932,Нижний Ландех,m
4,КЮН-1980-Юж-2018,Колесова Юлия Николаевна,1980,Г. Южа,f


In [130]:
d_inf = {}
k = 250
for i in range(len(informators)):
    d_inf[informators.iloc[i]['шифр информанта']] = k + i

In [131]:
d_inf

{'АНВ-1958-НЛ-2018': 250,
 'АН-1940-Юж-2018': 251,
 'ДЛЮ-1958-МН-2018': 252,
 'ЖЕА-1932-НЛ-2018': 253,
 'КЮН-1980-Юж-2018': 254,
 'КЕП-1958-Не-2017': 255,
 'КЛД-1947-Ро-2017': 256,
 'КЛД-1947-Ро-2017-2': 257,
 'КЕМ-1935-Лу-2018': 258,
 'КНА-1940-Не-2017': 259,
 'ЛИФ-1937-Му-2018': 260,
 'ЛИС-1935-Лу-2018': 261,
 'МСБ-1952-МН-2018': 262,
 'МВН-1958-Му-2018': 263,
 'МНП-1932-Юж-2018': 264,
 'НАЛ-1950-МН-2018': 265,
 'НА-1940-Юж-2018': 266,
 'ОРА-1939-Не-2017': 267,
 'ПЗИ-1948-Не-2017': 268,
 'ПАА-1946-НЛ-2018': 269,
 'ПЗЛ-1940-Вы-2017': 270,
 'РНА-1940-НЛ-2018': 271,
 'РЕ-1940-НЛ-2018': 272,
 'РРМ-1955-Не-2017': 273,
 'Се': 274,
 'САП-1924-Му-2018': 275,
 'СВВ-1956-Му-2018': 276,
 'СМП-1945-Не-2017': 277,
 'СЗТ-1930-Пр-2017': 278,
 'СНП-1975-Му-2018': 279,
 'СРВ-1935-МН-2018': 280,
 'ТГП-1930-НЛ-2018': 281,
 'ФНА-1931-Лу-2018': 282,
 'ЧЛВ-1983-Не-2017': 283,
 'ШНМ-1927-Му-2018': 284,
 'ШНН-1957-МН-2018': 285,
 'ШАВ-1938-Юж-2018': 286,
 'АВА-1949-НЛ-2018': 287,
 'КЛИ-1941-НЛ-2018': 288,
 

In [132]:
for i in range(len(informators)):
    row = informators.iloc[i]
    cur.execute('INSERT INTO informators (old_id, name, birth_year, birth_village, gender) \
    VALUES (%s, %s, %s, %s, %s )', (row['шифр информанта'],
                                   row['ФИО полное'],
                                   int(row['год рождения']),
                                   row['место рождения'],
                                   row['gender'],))

In [136]:
for i in range(len(informators)):
    row = informators.iloc[i]
    cur.execute('UPDATE informators SET code=%s WHERE id=%s', (row['шифр информанта'].split('-')[0], k+i,))

In [137]:
db.commit()

In [138]:
collectors = pd.read_csv('./csv/collectors.tsv', sep='\t').fillna('')

In [142]:
collectors.head()

,код,имя
0,АНВ,Агеичева Нина Владиславовна
1,ААЯ,Алина Алексеевна Яськова
2,БПА,Брюхачева Полина Александровна
3,БДД,Бударова Дарья Дмитриевна
4,ГНА,Григорьева Надежда Александровна


In [140]:
d_collectors = {}
k = 30
for i in range(len(collectors)):
    d_collectors[collectors.iloc[i]['код']] = k + i

In [141]:
d_collectors

{'АНВ': 30,
 'ААЯ': 31,
 'БПА': 32,
 'БДД': 33,
 'ГНА': 34,
 'ДВА': 35,
 'ЗАА': 36,
 'КЛИ': 37,
 'КМП': 38,
 'КАГ': 40,
 'КЮМ': 41,
 'МНМ': 42,
 'ПМА': 43,
 'ТОВ': 44,
 'ТСХ': 45,
 'ФВП': 46,
 'ХТС': 47,
 'ШДВ': 48,
 'ЮМК': 49,
 'НПЛ': 50,
 'НИВ': 51,
 'ЯАА': 52,
 'БМВ': 53}

In [145]:
for i in range(len(collectors)):
    row = collectors.iloc[i]
    cur.execute('INSERT INTO collectors (code, name) VALUES (%s, %s)', (row['код'], row['имя'],))

In [146]:
db.commit()

# A

In [151]:
data = pd.read_csv('./csv/A.tsv', sep='\t').fillna('')

In [152]:
data.head(2)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели
0,Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 300,2018,Рассказ о прошлом. Рассказ о хоре. Репертуар.,"устная история, песни, молодой-старый",,,0:25,3:00,Ивановская область,Пестяковский район,с. Нижний Ландех,Дом культуры и досуга Нижнеландеховского сельс...,"АВА-1949-НЛ-2018, КЛИ-1941-НЛ-2018, КВН-1951-Н...",КЮМ
1,Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 301,2018,Лирическая песня. Протяжная. Сведения о песне,"песни, лебедь, перо, девушка",,,3:01,8:20,Ивановская область,Пестяковский район,с. Нижний Ландех,Дом культуры и досуга Нижнеландеховского сельс...,"АВА-1949-НЛ-2018, КЛИ-1941-НЛ-2018, КВН-1951-Н...",КЮМ


In [156]:
parsed = []
for row in range(len(data)):
    i = data.iloc[row]
    parsed.append(
        (i['id'],int(i['год']),i['жанр'],i['время начала'],i['время конца'],
         i['регион'],i['район'],i['деревня'],i['адрес'],)
    )

In [157]:
cur.executemany('INSERT INTO texts (old_id, year, genre, start_text, finish_text, \
region, district, village, address) \
VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s)', parsed)

In [158]:
db.commit()

In [162]:
loc_inf = []
k = 4132
for row in range(len(data)):
    i = data.iloc[row]
    #print (i['информанты'].split(','))
    for j in i['информанты'].split(','):
        loc_inf.append((row+k,d_inf[j.strip()],))

In [164]:
cur.executemany('INSERT into t_i (id_text, id_informator) VALUES (%s, %s)', loc_inf)

In [165]:
db.commit()

In [166]:
loc_col = []
k = 4132
for row in range(len(data)):
    i = data.iloc[row]
    for j in i['собиратели'].split(','):
        loc_col.append((row+k,d_collectors[j.strip()],))
cur.executemany('INSERT into t_c (id_text, id_collector) VALUES (%s, %s)', loc_col)

In [167]:
db.commit()

# M

In [171]:
data = pd.read_csv('./csv/M.tsv', sep='\t').fillna('')
data.head(2)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели
0,Архив ВШЭ – Э – 37 – 20 – 2018 – 400,2018,Легенда. Об отпечатавшемся образе,"легенды, икона, церковь, народное православие,...",,,6:05,7:13,Ивановская область,Южский район,с. Мугреево-Никольское,"с. Мугреево-Никольское, в храме Артемия Веркол...",МСБ-1952-МН-2018,КЮМ
1,Архив ВШЭ – Э – 37 – 20 – 2018 – 401,2018,Легенда. О свете в храме,"легенды, церковь, пение, народное православие,...",,,7:13,9:55,Ивановская область,Южский район,с. Мугреево-Никольское,"с. Мугреево-Никольское, в храме Артемия Веркол...",МСБ-1952-МН-2018,КЮМ


In [172]:
parsed = []
for row in range(len(data)):
    i = data.iloc[row]
    parsed.append(
        (i['id'],int(i['год']),i['жанр'],i['время начала'],i['время конца'],
         i['регион'],i['район'],i['деревня'],i['адрес'],)
    )
cur.executemany('INSERT INTO texts (old_id, year, genre, start_text, finish_text, \
region, district, village, address) \
VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s)', parsed)

In [173]:
db.commit()

In [174]:
loc_inf = []
k = 4168
for row in range(len(data)):
    i = data.iloc[row]
    #print (i['информанты'].split(','))
    for j in i['информанты'].split(','):
        loc_inf.append((row+k,d_inf[j.strip()],))
cur.executemany('INSERT into t_i (id_text, id_informator) VALUES (%s, %s)', loc_inf)

In [175]:
db.commit()

In [176]:
loc_col = []
k = 4168
for row in range(len(data)):
    i = data.iloc[row]
    for j in i['собиратели'].split(','):
        loc_col.append((row+k,d_collectors[j.strip()],))
cur.executemany('INSERT into t_c (id_text, id_collector) VALUES (%s, %s)', loc_col)

In [177]:
db.commit()

# N

In [178]:
data = pd.read_csv('./csv/N.tsv', sep='\t').fillna('')
data.head(2)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели
0,Архив ВШЭ – ФЭ – 69 – 14 – 2017 – 600,2018,Календарный обряд.Святки,"календарь,праздник,Святки",,,2:52:33,2:53:19,Тверская область,Кимрский район,д.Высоково,музей крестьянского быта с. Неклюдово,ПЗЛ-1940-Вы-2017,"НПЛ, НИВ"
1,Архив ВШЭ – ФЭ – 69 – 14 – 2017 – 601,2018,Календарный обряд.Пасха,"календарь,праздник,Пасха",,,2:57:01,2:59:45,Тверская область,Кимрский район,д.Высоково,музей крестьянского быта с. Неклюдово,ПЗЛ-1940-Вы-2017,"НПЛ, НИВ"


In [179]:
parsed = []
for row in range(len(data)):
    i = data.iloc[row]
    parsed.append(
        (i['id'],int(i['год']),i['жанр'],i['время начала'],i['время конца'],
         i['регион'],i['район'],i['деревня'],i['адрес'],)
    )
cur.executemany('INSERT INTO texts (old_id, year, genre, start_text, finish_text, \
region, district, village, address) \
VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s)', parsed)

In [180]:
db.commit()

In [181]:
loc_inf = []
k = 4267
for row in range(len(data)):
    i = data.iloc[row]
    #print (i['информанты'].split(','))
    for j in i['информанты'].split(','):
        loc_inf.append((row+k,d_inf[j.strip()],))
cur.executemany('INSERT into t_i (id_text, id_informator) VALUES (%s, %s)', loc_inf)

In [182]:
db.commit()

In [ ]:
loc_col = []
k = 4267
for row in range(len(data)):
    i = data.iloc[row]
    for j in i['собиратели'].split(','):
        loc_col.append((row+k,d_collectors[j.strip()],))
cur.executemany('INSERT into t_c (id_text, id_collector) VALUES (%s, %s)', loc_col)

In [ ]:
db.commit()

# Ns

In [186]:
data = pd.read_csv('./csv/Ns.tsv', sep='\t').fillna('')
data.head(2)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели
0,Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 1,2018,"Похоронный обряд. Вынос из дома, дорога до кла...","гроб, покойник, похоронный обряд, дорога, проц...",,,0:00,1:47,Ивановская область,Южский район,г. Южа,"Г. Южа, дом ремесел",КЮН-1980-Юж-2018,КЮМ
1,Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 2,2018,"Календарные праздники. Пасха. Обрядовая пища, ...","яйцо пасх., церковь, освящение, оберег, пожар,...",,,1:47,4:55,Ивановская область,Южский район,г. Южа,"Г. Южа, дом ремесел",КЮН-1980-Юж-2018,КЮМ


In [187]:
parsed = []
for row in range(len(data)):
    i = data.iloc[row]
    parsed.append(
        (i['id'],int(i['год']),i['жанр'],i['время начала'],i['время конца'],
         i['регион'],i['район'],i['деревня'],i['адрес'],)
    )
cur.executemany('INSERT INTO texts (old_id, year, genre, start_text, finish_text, \
region, district, village, address) \
VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s)', parsed)

In [188]:
db.commit()

In [189]:
loc_inf = []
k = 4575
for row in range(len(data)):
    i = data.iloc[row]
    #print (i['информанты'].split(','))
    for j in i['информанты'].split(','):
        loc_inf.append((row+k,d_inf[j.strip()],))
cur.executemany('INSERT into t_i (id_text, id_informator) VALUES (%s, %s)', loc_inf)

In [190]:
db.commit()

In [191]:
loc_col = []
k = 4575
for row in range(len(data)):
    i = data.iloc[row]
    for j in i['собиратели'].split(','):
        loc_col.append((row+k,d_collectors[j.strip()],))
cur.executemany('INSERT into t_c (id_text, id_collector) VALUES (%s, %s)', loc_col)

In [192]:
db.commit()

# Тексты

In [4]:
import os

In [5]:
filelist = os.listdir('./txt/')

In [6]:
name = 'Архив ВШЭ – Э – 37 – 20 – 2018 – 456.txt'

In [6]:
def split_text(text):
    result = re.findall('^(.*?Ключевые.*?\n[\nА-Яа-я0-9\- ;]+?\n)(.*?)$', text, re.DOTALL)
    return result

In [7]:
def find_keywords(meta):
    result = [i.strip().capitalize() for i in re.findall('Ключевые слова:(.*?)\n', meta)[0].split(',')]
    if len(result) == 1 and len(result[0])<2:
        return None
    else:
        return result

In [35]:
find_keywords('Календарный обряд. Ильин день. Духов день. Троица \n32:10-32:54\nКлючевые слова: календарь, купание, запрет, Ильин день, Духов день, Троица\nСРВ-1935-МН-2018')

['Календарь', 'Купание', 'Запрет', 'Ильин день', 'Духов день', 'Троица']

In [10]:
for name in filelist:
    with open ('./txt/'+name) as f:
            text = f.read()
            result = split_text(text)
            if find_keywords(result[0]) == False:
                print (name)
            

Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 36.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 69.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 67.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 40.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 43.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 45.txt


In [262]:
with open ('./txt/Архив ВШЭ – Э – 37 – 20 – 2018 – 434.txt') as f:
    print (f.read())

Календарный обряд. Ильин день. Духов день. Троица 
32:10-32:54
Ключевые слова: календарь, купание, запрет, Ильин день, Духов день, Троица
СРВ-1935-МН-2018

С: Купаться ходили? 
РВ: Ну, у нас пруды были. Ходили, конечно, купались.
С: А когда начинали? Приметы были какие-то? 
РВ: А вот, наверно, с Духова дня, начинали купаться, а в Ильин день кончали. Ильин день 2 августа, по моему. А Троица, она же не в числах. Когда Духов день.
С: В Ильин день уже нельзя было купаться или еще…
РВ: Ну, раньше-то мы как-то не купались, а теперь не обращают внимания, купаются. 



In [263]:
with open ('./txt/Архив ВШЭ – Э – 37 – 20 – 2018 – 434.txt') as f:
    print (split_text(f.read()))

[('Календарный обряд. Ильин день. Духов день. Троица \n32:10-32:54\nКлючевые слова: календарь, купание, запрет, Ильин день, Духов день, Троица\nСРВ-1935-МН-2018\n', '\nС: Купаться ходили? \nРВ: Ну, у нас пруды были. Ходили, конечно, купались.\nС: А когда начинали? Приметы были какие-то? \nРВ: А вот, наверно, с Духова дня, начинали купаться, а в Ильин день кончали. Ильин день 2 августа, по моему. А Троица, она же не в числах. Когда Духов день.\nС: В Ильин день уже нельзя было купаться или еще…\nРВ: Ну, раньше-то мы как-то не купались, а теперь не обращают внимания, купаются. ')]


In [278]:
for name in filelist:
    cur.execute('SELECT id FROM texts WHERE old_id=%s', (name.replace('.txt',''),))
    id_text = cur.fetchall()[0][0]
    with open ('./txt/'+name) as f:
        text = split_text(f.read())[0][1]
        cur.execute('UPDATE texts SET raw_text=%s WHERE id=%s', (text, id_text))

In [279]:
db.commit()

# Ключевые слова

In [20]:
data = pd.read_csv('/home/dkbrz/GitHub/folklore_corpus/csv/prev/tblCards.csv', sep='\t').fillna('')

In [13]:
data.shape

(4131, 21)

In [21]:
data.head(2)

,текст,аннотация,ключевые слова,термины,программа,вопрос,село,год,информант,информант2,...,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер,zvuk,zvuk2,photo,photo2
0,"[Шёл разговор о пьющей женщине из Карпиловки, ...",,"Водка, Плата, Питье, Девственность, Женщина, П...",,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,,МЭГ,НВП,СЕ,ФЕЕ,1,,,,
1,[Разговор о кусачей собаке:] [Г:] А раз её\ э\...,,"Первый, Прозвище, Имя, Еврей, Народ, Свой-чужой",,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,,МЭГ,НВП,СЕ,ФЕЕ,2,,,,


In [32]:
cur.execute('SELECT id, word FROM keywords')
d_key = {}
for i in cur.fetchall():
    d_key[i[1]] = i[0]

In [37]:
for i in range(len(data)):
    row = data.iloc[i]
    keywords = list(set([word.strip() for word in row['ключевые слова'].split(',')]))
    for word in keywords:
        if word in s:
            cur.execute('INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)', (i+1, d_key[word],))

In [38]:
db.commit()

In [31]:
found -(found&s) - set(["Вкус","Гадюка","Глина","Двенадцать","Десять","Звон","Каштан","Кнут","Красота","Обряд","Помидор","Пять","Ребро","Рогоз","Сабельник","Сапоги","Свекла","Смородина","Сто","Теленок","Туман","Улица","Чай","Четыре","Читать"])

{'', 'Варвара св.', 'Давать-брать', 'Фома св.'}

In [10]:
misspell = {"Благословение":"Благословлять","Быличка":"Былички","Вербное воскресенье":"Вербное воскресение","Воскресенье":"Воскресенье (день недели)","Гад":"Гады","Гадание":"Гадания","Гуляние":"Гулять","Гулянь":"Гулять","Духовные стихи.":"Духовные стихи","Жаворонки":"Жаворонок","Жена":"Супруги","Заговор":"Заговоры","Запрет":"Запреты","Знахари":"Знахарь","Иван купала":"Иван Купала","Иконы":"Икона","Каравай":"Коровай","Кикиимора":"Кикимора","Кладище":"Кладбище","Колдуны":"Колдун","Красный":"Красный цвет","Крещение":"Крещенье (праздник)","Крещенье":"Крещенье (праздник)","Легенда":"Легенды","Музыкальный инструмент":"Музыкальные инструменты","Мыть":"Мытье","Новороденный":"Новорожденный","Оберег":"Обереги","Обмывание":"Обмывание покойника","Осыпать":"Осыпать","Песня":"Песни","Петр и павел":"Петр и Павел","Петра и павла день":"Петра и Павла день","Петух 31:07-36:26":"Петух","Покойники":"Покойник","Покров":"Покров (календ.)","Поминальный дни":"Поминальные дни","Праздник":"Праздники","Предание":"Предания","Предание топоним.":"Предания топоним.","Предания топоним":"Предания топоним.","Приговор":"Приговоры","Примета":"Приметы","Прозвища":"Прозвище","Птица":"Птицы","Радоница":"Радуница","Рождество богородицы":"Рождество Богородицы","Ряжение":"Ряженье","Свечаэ=":"Свеча","Святой":"Святые","Смерть":"Жизнь-смерть","Сновидение":"Сновидения","Соседи":"Сосед","Танцевать":"Танец","Угощать":"Угощение","Урожай":"Урожай-неурожай","Успение богородицы":"Успение Богородицы","Шутка":"Шутки","Яйца":"Яйцо"}

In [48]:
not_found = set()
for name in filelist:
    cur.execute('SELECT id FROM texts WHERE old_id=%s', (name.replace('.txt',''),))
    id_text = cur.fetchall()[0][0]
    with open ('./txt/'+name) as f:
            keywords = find_keywords(split_text(f.read())[0][0])
            if keywords is not None:
                for word in keywords:
                    if word not in s:
                        if word in misspell:
                            word = misspell[word]

In [49]:
for name in filelist:
    cur.execute('SELECT id FROM texts WHERE old_id=%s', (name.replace('.txt',''),))
    id_text = cur.fetchall()[0][0]
    with open ('./txt/'+name) as f:
            keywords = find_keywords(split_text(f.read())[0][0])
            if keywords is not None:
                for word in keywords:
                    try:
                        if word != '':
                            if word in misspell:
                                word = misspell[word]
                                print ('|',end='')
                            if word in s:
                                cur.execute('SELECT id FROM keywords WHERE word = %s', (word,))
                                id_word = cur.fetchone()[0]
                                cur.execute('INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)', (id_text, id_word,))
                    except:
                        print(word)

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [9]:
cur.execute('SELECT DISTINCT word FROM keywords')
s = set()
for i in sorted(set([i[0] for i in cur.fetchall()])):
    s.add(i)

In [27]:
misspell2 = {"Бывальщина":"Былички","Егорьев день":"Юрьев день","Егорьер день":"Юрьев день","Застолье":"Трапеза ритуальная","Знамение":"Знак-знамение","Игры":"Игры фольк.","Коса":"Коса (волосы)","Лук":"Лук (раст.)","Мальчишник":"Девишник","Молодые":"Молодожены","Муж":"Супруги","Обрядовое деревце":"Деревце обрядовое","Петровский обряд":"Петра и Павла день","Плач":"Причитания","Поминание":"Поминальные дни","хороны":"Похоронный обряд","Рассказ о прошлом":"Устная история","Ритуальная речь":"Говорить","Родственники":"Родство","Ряженые":"Ряженье","Ряженый":"Ряженье","Свадебный":"Свадебный обряд","Свадьба":"Свадебный обряд","Сваты":["Сват", "Сваха"], "Гора, лес":["Гора", "Лес"], "Семья":["Родство","Супруги"],"Стихи":"Народная литература","Частушка":"Песни"}

In [29]:
for name in filelist:
    cur.execute('SELECT id FROM texts WHERE old_id=%s', (name.replace('.txt',''),))
    id_text = cur.fetchall()[0][0]
    with open ('./txt/'+name) as f:
            keywords = find_keywords(split_text(f.read())[0][0])
            if keywords is not None:
                for word in keywords:
                    try:
                    #if True:
                        if word != '':
                            if (not word in misspell) and (not word in s):
                                if word in misspell2:
                                    if type(misspell2[word]) != list:
                                        if misspell2[word] in s:
                                            cur.execute('SELECT id FROM keywords WHERE word = %s', (misspell2[word],))
                                            id_word = cur.fetchone()[0]
                                            cur.execute('INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)', (id_text, id_word,))
                                    else:
                                        for w in misspell2[word]:
                                            cur.execute('SELECT id FROM keywords WHERE word = %s', (w,))
                                            id_word = cur.fetchone()[0]
                                            cur.execute('INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)', (id_text, id_word,))
                    except:
                        print(word)

In [30]:
db.commit()

# Fixes

In [32]:
cur.execute('SELECT region, id FROM texts')
data = cur.fetchall()
for i in data:
    if 'обл.' in i[0]:
        new = i[0].replace('обл.','область')
        cur.execute('UPDATE texts SET region=%s WHERE id = %s', (new, i[1]))

In [34]:
cur.execute('SELECT district, id FROM texts')
data = cur.fetchall()
for i in data:
    if 'р-н' in i[0]:
        new = i[0].replace('р-н','район')
        cur.execute('UPDATE texts SET district=%s WHERE id = %s', (new, i[1]))

In [57]:
db.commit()

In [40]:
cur.execute('SELECT village, id FROM texts')
data = cur.fetchall()
for i in data:
    if '.' in i[0]:
        if len(i[0]) < 30:
            new = i[0].split('.')[1].strip()
            cur.execute('UPDATE texts SET village=%s WHERE id = %s', (new, i[1]))

In [55]:
video = {"Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 109":"WTA5A1DE_M0",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 110":"wMghKiSvZkU",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 112":"wAH27oKlFus",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 113":"QxNt-Xnpm4s",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 114":"vNTrN7YS0s8",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 115":"FNbjE2iR5ns",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 116":"gHKWxfYAVR4",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 117":"AQPzilvewjo",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 303":"a9I-emOXnl0",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 304":"a9aPIdPt1AM",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 305":"N9gVWcywOps",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 306":"Gp7JUuCw6Kc",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 311":"EJ9PLvrSwdg",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 312":"ksFPLPooIjg",
         "Архив ВШЭ – ФЭ – 37 – 12 – 2018 – 313":"pvvK4ZcwAVE"}

In [58]:
video = {"3888":"G60R8AfP4JQ\n0xks23g-zxo	","2449":"n-PbiQaAN-w","2448":"8vop3TONDCk","2446":"tDrlvI_cyzs","2427":"gipJfuGFkx8","2426":"c6FQQyJJY-s","2425":"0wnPouXWimg","2524":"UlmJaMy87nM","2423":"qO533x_parc","2422":"JgRk2AS9F_Y","2420":"ZGP03YEYd8s","2415":"_E26Wu-RX3w","2414":"VThjHM1-iHU","2413":"ZbRy6g3B-yk","2412":"sgmg7lmMbzU","2411":"rUK3OlyEeYo","2410":"DmZWW45VeXw","2409":"MyZHSEUeUb8\n_S_fdEFcRa0\nW-OtlWXfpuY","1697":"6k1m8IyltYg"}

In [61]:
for name in video:
    cur.execute('SELECT id FROM texts WHERE old_id=%s', (name,))
    id_text = cur.fetchall()[0][0]
    cur.execute('UPDATE texts SET video=%s WHERE id = %s', (video[name], id_text))
    print (id_text)

3884
2465
2464
2462
2443
2442
2441
2540
2439
2438
2436
2431
2430
2429
2428
2427
2426
2425
1751


In [62]:
db.commit()

In [54]:
'12233'.split('\n')

['12233']